<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week3/Autonomous_driving_application_Car_detection_v3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autonomus driving - Car detection #

3주차 프로그래밍 과제에 오신것을 환영합니다! 여러분은 지금부터 강력한 YOLO 모델을 사용한 객체 감지 알고리즘을 구현할 것입니다. 이 과제 노트북의 주된 아이디어들은 YOLO 알고리즘으 다룬 두 논문 [Redmon et al., 2016](https://arxiv.org/abs/1506.02640), [Redmon and Farhadi, 2016](https://arxiv.org/abs/1612.08242) 에서 차용했습니다. 

**이번 과제에서 여러분은**:
- 자동차 감지 데이터셋에 대해 객체 감지 알고리즘을 사용할 수 있습니다.
- 경계 상자(bounding box)를 다룰 수 있습니다.

## Import libraries ##

아래 코드 블록을 실행시켜 객체 감지 알고리즘을 만들 때  필요한 패키지와 의존성들을 불러오세요.

In [ ]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from yad2k.models.keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

%matplotlib inline

**중요 참고 사항**: 위에서 볼 수 있듯이, Keras 백엔드를 K라는 이름으로 불러왔습니다. 이는 Keras 함수를 `K.function(...)`과 같은 형태로 호출하여 사용할 수 있다는 것을 의미합니다.

## 1 - Problem statement ##

여러분은 현재 자율 주행 차를 만드는 회사에서 일하고 있습니다. 이 프로젝트의 가장 중요한 기능으로, 여러분은 먼저 자동차 감지 시스템을 만들어야 합니다. 데이터를 수집하기 위해서 여러분은 자동차 지붕에 카메라를 장착하여 운전하는 동안 몇 초마다 전방 도로 사진을 찍었습니다.


<video width="400" height="200" src="arts/road_video_compressed2.mp4" type="video/mp4" controls>
</video>

위 영상은 [drive.ai](https://www.drive.ai/) 에서 수집한 자동차 지붕에 달린 카메라로 촬영한 실리콘밸리 도로 영상입니다.

이 모든 이미지를 폴더에 모아서 찾은 모든 자동차 주위에 bounding box를 그려서 라벨을 지정했습니다. 다음은 bounding box의 모양에 대한 예입니다.

<br>

<img src="arts/box_label.png" style="width:500px;height:250;">
<center><u><b>그림 1</b></u>: <b>box의 정의</b></center>

객체 감지기가 인식하도록 하려는 80 개의 클래스가 있는 경우, 클래스 라벨 c는 1에서 80까지의 정수 또는 값 하나가 1이고 나머지는 0인 80 차원 벡터로 나타낼 수 있습니다. 지난 영상 강의에서는 후자의 표현을 사용했습니다. 이 과제에서는 특정 단계에 맞게 더 편리하게 적용할 수 있도록 두 표현 방법을 다 사용해 보겠습니다.

이 과제에서는 "You Only Look Once"(YOLO)알고리즘으로 물체 감지를 수행하는 방법을 배우고 이를 자동차 감지에 적용해봅니다. YOLO 모델은 학습하는 데 계산 비용이 많이 들기 때문에 사전 학습된(pre-trained) 가중치를 사용할 수 있습니다.


## 2 - YOLO ##

"You Only Look Once"(YOLO)는 높은 정확도를 달성하면서도 실시간으로 실행할 수 있기 때문에 인기있는 객체 검출 알고리즘입니다. 이 알고리즘은 예측을 위해 네트워크를 통해 단 하나의 순방향 전파가 필요하다는 점에서 이미지를 "한 번만 본다(You only look once)"라고 표현할 수 있습니다. non-max suppression 이후 bounding box와 함께 인식 된 객체가 무엇인지 출력합니다.

### 2.1 - Model details ###

#### Inputs and outputs
- **입력**은 배치 사이즈를 m으로 하는 (m, 608, 608, 3) shape의 이미지 배치입니다.
- **출력**은 인식된 클래스와 그 객체의 bounding box입니다. 각 bounding box는 위 그림에 나타난 ($p_c, b_x, b_y, b_h, b_w, c)의 6개의 숫자로 표현될 수 있습니다. 만약 c를 80차원의 벡터로 확장하면, 각 bounding box는 85개의 숫자로 표현할 수 있습니다.

<br>

#### Anchor Boxes
- 서로 다른 클래스를 나타내는 합리적인 높이.너비비를 선택하기 위해서 훈련 세트를 탐색하면서 anchor box를 찾습니다. 이번 과제에서는 80개의 서로 다른 클래스를 커버하기 위해서 5개의 anchor box가 사용됩니다. 이 값은 `./model_data/yolo_anchors.txt` 파일에 저장되어 있습니다.
- anchor box의 차원은 encoding ($m, n_H, n_y, anchors, classes) 에서 마지막 두 개의 값입니다.
- YOLO 아키텍쳐는 IMAGE(m, 608, 608, 3) -> DEEP CNN -> ENCODING(m, 19, 19, 5, 85) 로 이루어져 있습니다.

<br>

#### Encoding

ENCODING 파트가 무엇을 하는지 좀 더 자세히 알아봅시다.

<img src="arts/architecture.png" style="width:700px;height:400;">
<center><u><b>그림 2</b></u>: <b>YOLO 알고리즘의 Encoding 아키텍쳐</b></center>

감지하고자 하는 객체의 중심/중간 점이 그리드 셀에 속하면 해당 그리드 셀이 해당 객체를 감지합니다.

우리는 5개의 anchor box를 사용하기 때문에, 19x19 셀은 5개의 박스에 대한 각각의 정보를 가지고 있습니다. anchor box들은 width, height으로만 정의합니다.

편의를 위해 (19, 19, 5, 85) shape의 마지막 두 차원을 하나로 평면화시키겠습니다. 따라서 Deep CNN 블록의 최종 출력값은 (19, 19, 425) 가 됩니다.

<img src="arts/flatten.png" style="width:700px;height:400;">
<center><u><b>그림 3</b></u>: 마지막 두 차원을 평면화하기</center>


#### Class score

이제, 각 셀에 속해있는 각각의 박스에 대해 아래의 element-wise 곱셈 연산을 취하고 해당 박스가 특정 클래스의 물체를 포함하고 있는지에 대한 확률을 추출해보겠습니다.

class score는 $score_{c, i} = p_c \times c_i$ 의 공식으로 나타낼 수 있습니다. 이는 box 내에 물체가 있을 확률 $p_c$과 그 물체가 특정 클래스 $c$일 확률 $c_i$를 곱한 값입니다. 

<img src="arts/probability_extraction.png" style="width:700px;height:400;">
<center><b>그림 4</b>: 각 경계 상자로부터 물체의 클래스를 찾아내기</center>


#### *Example of 그림 4*
- 그림 4에서, 1번 셀의 1번 box를 들여다봅시다. 해당 박스 안에 물체가 있을 확률 $p_1 = 0.60$입니다. 따라서 cell 1의 box 1번에 물체가 존재할 확률은 60%입니다.
- box 1번의 물체가 3번 카테고리일 확률 $c_3 = 0.73$입니다.
- 따라서 box1에 대한 "3"번 카테고리의 score는, $score_{1, 3} = 0.60 \times 0.73 = 0.44$ 로 나타낼 수 있습니다.
- box 1의 80 개 클래스에 대한 점수를 모두 계산하고 자동차 클래스(클래스 3)의 점수가 최대 값임을 확인한다고 가정 해 보겠습니다. 따라서 이 box "1"에 점수 0.44와 클래스 "3"을 할당합니다.


#### Visualizing classes

다음은 YOLO 알고리즘이 이미지에서 예측한 결과를 시각화하는 한 가지 방법입니다.
- 19x19 그리드 셀 각각에 대해 확률 점수의 최대 값을 찾습니다 (80 개 클래스에서 최대 값, 5 개의 앵커 상자 각각에 대해 최대 값).
- 그리드 셀이 가장 가능성이 높은 것으로 간주하는 객체에 따라 그리드 셀에 색상을 지정합니다.

위 작업을 그림으로 나타내면 다음과 같습니다.

<img src="arts/proba_map.png" style="width:300px;height:300;">
<center><u><b>그림 5</b></u>: 19x19 그리드 셀 각각에 대해 가장 확률이 높다고 예측된 클래스로 색칠된 모습</center>

이 시각화는 예측을 위한 YOLO 알고리즘의 핵심 부분은 아닙니다. 알고리즘의 중간 결과를 시각화하는 좋은 방법입니다.

#### Visualizing bounding boxes

YOLO 알고리즘의 결과를 시각화하는 다른 방법은 bounding boxes를 그리는 것입니다. 이 시각화를 적용한 결과는 다음과 같습니다.

<img src="arts/anchor_map.png" style="width:200px;height:200;">
<center><u><b>그림 6</b></u>: 각각의 셀은 5개의 bounding box를 제공해줍니다. 총합하면 이 모델은 한 회의 정방향 연산에 대해 19x19x5 = 1805 개의 bounding box를 생성합니다. 서로 다른 색깔은 서로 다른 클래스를 의미합니다.</center>

<br>

#### Non-Max suppression

위 그림에서, 높은 확률을 가진 bounding box들 만을 간추려서 그렸음에도 불구하고 box가 매우 많습니다. 여러분은 알고리즘이 출력한 감지된 객체를 훨씬 적은 수로 줄여야 합니다.

이를 위해서, **non-max suppression**을 사용합니다. 구체적으로, 아래 단계를 따릅니다.
- 적은 score를 가지고 있는 박스를 없앱니다 (이는, 해당 박스가 객체 감지에 그다지 자신이 없다는 것을 의미합니다. 물체가 존재할 확률 자체가 낮거나, 그 물체가 특정 클래스에 속할 확률이 낮기 때문입니다).
- 여러 상자가 서로 겹치는 경우 하나의 상자 만 선택하고 동일한 개체를 감지합니다.




### 2.2 - Filtering with a threshold on class scores ###

먼저 임계치을 사용하여 필터를 적용합니다. class score가 선택한 임계 값보다 작은 상자를 제거하려고 합니다.

이 모델은 총 19x19x5x85 숫자를 제공하며 각 상자는 85 개의 숫자로 설명됩니다. (19,19,5,85) 혹은 (19,19,425) 차원 텐서를 다음 변수로 reshape하는 것이 좋을 것 같습니다.
- `box_confidence`: ($19 \times 19, 5, 1$)의 shape를 가진 텐서로, 셀에서 예측된 5개의 box 각각에 대한 $p_c$(특정 물체가 있을 확률)를 포함합니다.
- `boxes`: ($19 \times 19, 5, 4$)의 shape인 텐서로, 5개의 box의 중심점과 넓이에 대한 정보인 ($b_x, b_y, b_h, b_w$)를 가지고 있습니다.
- `box_class_probs`: ($19 \times 19, 5, 80$)의 shape를 가지고 있는 텐서로, ($c_1, c_2, ..., c_{80}$) 까지의 각 클래스에 대한 확률을 담고 있습니다.

#### 연습 문제: `yolo_filter_boxes()` 함수를 구현하세요
1. 그림 4에 나왔던 공식대로($p \times c$) 적절한 텐서를 elementwise 곱셈하여 box score를 계산하세요. 아래 코드가 도움이 될 수 있습니다.
```python
a = np.random.randn(19*19, 5, 1)
b = np.random.randn(19*19, 5, 80)
c = a * b # shape of c will be (19*19, 5, 80)
```
이 연산은 파이썬 **broadcasting**(서로 다른 사이즈의 벡터를 곱하기)의 예시입니다.
2. 각 box에 대해서, 아래의 두 값을 구해보세요.
  - 최댓값인 box score에 대한 class index
  - 그에 상응하는 box score

  **참조하면 좋은 것들**
    * [Keras argmax](https://keras.io/backend/#argmax)
    * [Keras max](https://keras.io/backend/#max)
  
  **추가적인 힌트**
    * `argmax`와 `max` 함수의 파라미터 `axis`에 대하여, 만약 여러분이 마지막 axis를 선택하려는 경우, 이를 수행하는 한 가지 방법은 `axis = -1`로 설정하는 것입니다. 이는 `arrayname[-1]`을 사용하여 배열의 마지막 위치를 선택할 수 있는 Python 배열 인덱싱과 유사합니다.
    * `max`를 사용하면 일반적으로 최댓값이 적용된 축이 축소됩니다. `keepdims=False`는 기본 옵션으로, 해당 차원을 전부 제거하고 최댓값만 리턴합니다. 이번 과제에서도 최대값을 적용한 이후 마지막 차원을 유지할 필요가 없습니다.
    * 문서에는 `keras.backend.argmax`가 표시되어 있지만 `keras.argmax`를 사용하세요. 위와 마찬가지로 `keras.max`를 사용하세요.
3. 임계치를 사용하여 마스크를 만듭니다. 아래 코드를 확인하세요.
```python
mask = [0.9, 0.3, 0.4, 0.5, 0.1] < 0.4
print(mask)   # [False, True, False, False, True]
```
mask 배열에서, 유지하려는 box의 index에 대한 값은  true여야 합니다.
4. TensorFlow를 사용하여 마스크를`box_class_scores`, `boxes` 및 `box_classes`에 적용하여 원하지 않는 상자를 필터링합니다. 유지하려는 상자의 하위 집합 만 남겨 두어야합니다.

  **유용한 참조** :
     * [boolean mask](https://www.tensorflow.org/api_docs/python/tf/boolean_mask)

  **추가적인 힌트** :
     * `tf.boolean_mask`의 경우 기본 `axis = None` 을 유지할 수 있습니다.

**알림** : Keras 함수를 호출하려면`K.function (...)`을 사용해야합니다.

In [ ]:
# GRADED FUNCTION: yolo_filter_boxes

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Step 1: Compute box scores
    ### START CODE HERE ### (≈ 1 line)
    box_scores = None
    ### END CODE HERE ###
    
    # Step 2: Find the box_classes using the max box_scores, keep track of the corresponding score
    ### START CODE HERE ### (≈ 2 lines)
    box_classes = None
    box_class_scores = None
    ### END CODE HERE ###
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    ### START CODE HERE ### (≈ 1 line)
    filtering_mask = None
    ### END CODE HERE ###
    
    # Step 4: Apply the mask to box_class_scores, boxes and box_classes
    ### START CODE HERE ### (≈ 3 lines)
    scores = None
    boxes = None
    classes = None
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_a:
    box_confidence = tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([19, 19, 5, 4], mean=1, stddev=4, seed = 1)
    box_class_probs = tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.5)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.shape))
    print("boxes.shape = " + str(boxes.shape))
    print("classes.shape = " + str(classes.shape))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           10.7506
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [ 8.42653275  3.27136683 -0.5313437  -4.94137383]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           7
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (?,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (?, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (?,)
        </td>
    </tr>
</table>

**참고**: `yolo_filter_boxes` 를 테스트 할 때 난수를 사용하고 있습니다. 실제 데이터에서`box_class_probs`는 확률에 대해 0과 1 사이의 0이 아닌 값을 포함합니다. `boxes`의 좌표는 길이와 높이가 음수가 아니도록 선택됩니다.

### 2.3 - Non-max suppression ###

class score를 기준으로 필터링 한 이후에도 여전히 서로 겹치는 bounding box가 많습니다. 아래 그림에서 오른쪽 상자를 선택하기 위한 두 번째 필터는 NMS(non-maximum suppression)입니다.

<img src="arts/non-max-suppression.png" style="width:500px;height:400;">
<center><u><b>그림 7</b></u>: 이 예시에서, 모델이 3개의 차를 예측했지만 실제로는 모두 같은 자동차입니다. non-max suppression(NMS) 알고리즘을 통해서 3개의 경계 상자 중 가장 정확한(가장 높은 확률을 가진) 상자를 추출할 수 있습니다</center>

<br>

Non-max suppression 알고리즘은 굉장히 중요한 개념인 **"Intersection over Union" : IOU** 라는 개념을 사용합니다.

<img src="nb_images/iou.png" style="width:500px;height:400;">
<center><u><b>그림 8</b></u>: Intersection over Union의 정의</center>

<br>

**연습 문제**: **`iou()` 함수를 구현하세요. 몇가지 힌트를 참조하시면 좋습니다**
- 이 코드에서, 좌표 (0,0)을 이미지의 좌측 상단, (1,0)을 우측 상단, (1,1)을 우측 하단으로 하는 컨벤션을 사용합니다. 다시 말해 (0,0) 좌표에서 x가 증가할 수록 우측으로 이동하고, y가 증가할수록 하단으로 이동합니다.
- 이 연습문제 에서는 중간 점, 높이 및 너비 대신 왼쪽 위 ($x1, y1$)와 오른쪽 아래 ($x2, y2$)의 두 모서리를 사용하여 상자를 정의합니다. (이렇게하면 교차점을 좀 더 쉽게 계산할 수 있습니다.)
- 직사각형의 면적을 계산하려면 높이 ($y2−y1$)에 너비 ($x2−x1$)를 곱합니다.(($x1, y1$)은 왼쪽 상단이고 $x2, y2$는 오른쪽 하단이므로 이러한 차이는 음수가 아니어야합니다.
- 두 box가 **겹치는 부분(intersection)**을 찾으세요.
  - 이를 명확히 이해하기 위해 종이에 몇 가지 예시를 그려보세요.
  - 교차되는 면의 왼쪽 상단 모서리인 ($x_{i1}, y_{i1}$)는 두 경계 상자의 왼쪽 상단 모서리($x1, y1$)를 비교하여, x좌표가 오른쪽에 더 가깝고, y좌표가 바닥에 더 가까운 정점으로 구할 수 있습니다.
  - 마찬가지로 교차면의 오른쪽 하단 모서리 $x_{i2}, y_{i2}$)는 두 경계 상자의 우측 하단 모서리 ($x2, y2$)를 비교해 x좌표가 왼쪽에, y좌표가 위쪽에 더 가까운 정점을 선택하여 구할 수 있습니다.
  - 두 경계 상자 사이의 **교차면이 없을 수도 있습니다.** 위에서 계산한 교차면의 각 좌표가 교차 box의 우측 상단 혹은 좌측 하단일 경우, 이 현상을 감지할 수 있습니다. 또한, 높이 ($y2−y1$) 또는 너비 ($x2−x1$)중 적어도 하나가 음수 인 경우 교차면은 0입니다.
  - 두 상자는 가장자리 또는 꼭지점에서 교차 할 수 있으며, 이 경우 교차 영역은 여전히 ​​0입니다. 이것은 계산 된 교차점의 높이나 너비 (또는 둘 다)가 0 일 때 발생합니다.

**추가적인 힌트**

- `xi1` = **최대** 두 상자의 x1 좌표
- `yi1` = **최대** 두 상자의 y1 좌표
- `xi2` = **최소** 두 상자의 x2 좌표
- `yi2` = **최소** 두 상자의 y2 좌표
- `inter_area` = `max (height, 0)` 및 `max (width, 0)` 사용 가능


In [ ]:
# GRADED FUNCTION: iou

def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """

    # Assign variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2. Calculate its Area.
    ### START CODE HERE ### (≈ 7 lines)
    xi1 = None
    yi1 = None
    xi2 = None
    yi2 = None
    inter_width = None
    inter_height = None
    inter_area = None
    ### END CODE HERE ###    

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    ### START CODE HERE ### (≈ 3 lines)
    box1_area = None
    box2_area = None
    union_area = None
    ### END CODE HERE ###
    
    # compute the IoU
    ### START CODE HERE ### (≈ 1 line)
    iou = None
    ### END CODE HERE ###
    
    return iou

In [ ]:
## Test case 1: boxes intersect
box1 = (2, 1, 4, 3)
box2 = (1, 2, 3, 4) 
print("iou for intersecting boxes = " + str(iou(box1, box2)))

## Test case 2: boxes do not intersect
box1 = (1,2,3,4)
box2 = (5,6,7,8)
print("iou for non-intersecting boxes = " + str(iou(box1,box2)))

## Test case 3: boxes intersect at vertices only
box1 = (1,1,2,2)
box2 = (2,2,3,3)
print("iou for boxes that only touch at vertices = " + str(iou(box1,box2)))

## Test case 4: boxes intersect at edge only
box1 = (1,1,3,3)
box2 = (2,3,3,4)
print("iou for boxes that only touch at edges = " + str(iou(box1,box2)))

**모범 답안**:
```
iou for intersecting boxes = 0.14285714285714285
iou for non-intersecting boxes = 0.0
iou for boxes that only touch at vertices = 0.0
iou for boxes that only touch at edges = 0.0
```

#### YOLO non-max suppression ####

이제 non-max suppression을 구현할 차례입니다. 핵심 단계는 다음과 같습니다.
1. 가장 높은 점수를 받은 box를 선택하세요.
2. 해당 box와 다른 모든 box의 겹치는 부분을 계산하고, 서로 크게 겹치는 상자를 제거합니다 (`iou >= iou_threshold`)
3. 1단계로 돌아가서 현재 선택한 box보다 점수가 낮은 box가 없을 때 까지 반복합니다.

이 세 단계를 따르면 선택한 상자와 크게 겹치는 다른 상자들을 제거하고, 가장 최선의 경계 상자만을 남겨놓을 수 있습니다.

**연습 문제**: Tensorflow를 사용해서 `yolo_non_max_suppression()` 함수를 구현하세요. Tensorflow에는 non-max suppression을 위한 빌트인 함수가 미리 정의되어 있습니다. 따라서 여러분은 위에서 직접 구현한 `iou()` 함수를 사용하지 않아도 됩니다.

**참고 문헌**:
- [tf.image.non_max_suppression()](https://www.tensorflow.org/api_docs/python/tf/image/non_max_suppression)
```
tf.image.non_max_suppression(
    boxes,
    scores,
    max_output_size,
    iou_threshold=0.5,
    name=None
)
```
- 이 과제에 사용 된 tensorflow 버전에는 `score_threshold` 매개 변수가 없으므로 (최신 버전에 대한 문서에 표시됨) 이 값을 설정하면 오류 메시지가 표시됩니다.
- [K.gather()](https://www.tensorflow.org/api_docs/python/tf/keras/backend/gather) <br> 공식 문서에는 `tf.keras.backend.gather()` 힘수가 나와도, 이 과제에서는 `keras.gather()` 함수를 사용해야 합니다..    
```
keras.gather(
    reference,
    indices
)
```

In [ ]:
# GRADED FUNCTION: yolo_non_max_suppression

def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    ### START CODE HERE ### (≈ 1 line)
    nms_indices = None
    ### END CODE HERE ###
    
    # Use K.gather() to select only nms_indices from scores, boxes and classes
    ### START CODE HERE ### (≈ 3 lines)
    scores = None
    boxes = None
    classes = None
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    scores = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    boxes = tf.random_normal([54, 4], mean=1, stddev=4, seed = 1)
    classes = tf.random_normal([54,], mean=1, stddev=4, seed = 1)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           6.9384
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [-5.299932    3.13798141  4.45036697  0.95942086]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           -2.24527
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (10, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
</table>

### 2.4 Wrapping up the filtering ###

deep CNN(19x19x5x85 dimension encoding)의 출력값을 가져와 지금까지 구현 한 함수를 사용하여 모든 box를 필터링하는 함수를 구현해봅시다.

**연습 문제** : YOLO encoding 출력을 인자로 받아서 NMS와 score threshold를 사용하여 경계 상자를 필터링 하는 `yolo_eval()` 함수를 구현해봅시다. 마지막으로 알아두어야 할 몇 가지 디테일이 있습니다. 경계 상자를 나타내는 방법에는 여러가지가 있습니다. 모서리를 사용해서 표현하거나, 중간 점 및 높이/너비를 사용하는 방법 등등이 그 중 하나입니다. YOLO 알고리즘에선, 사전에 제공된 아래의 함수를 사용하여 둘 사이의 형식을 변환할 수 있습니다.
```python
boxes = yolo_boxes_to_corners(box_xy, box_wh) 
```

YOLO의 네트워크는 608x608 이미지에서 실행되도록 훈련되었습니다. 이 데이터를 다른 크기의 이미지 (예 : 자동차 감지 데이터 세트에 720x1280 이미지가있는 경우)에서 테스트하는 경우이 단계에서는 원래 720x1280 이미지 위에서도 동작할 수 있도록 상자의 크기를 조정합니다.

이 두 함수에 대해 걱정하지 마십시오. 언제 호출해야 하는지 알려 드리겠습니다.

In [ ]:
# GRADED FUNCTION: yolo_eval

def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """
    
    ### START CODE HERE ### 
    
    # Retrieve outputs of the YOLO model (≈1 line)
    box_confidence, box_xy, box_wh, box_class_probs = None

    # Convert boxes to be ready for filtering functions (convert boxes box_xy and box_wh to corner coordinates)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold (≈1 line)
    scores, boxes, classes = None
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with 
    # maximum number of boxes set to max_boxes and a threshold of iou_threshold (≈1 line)
    scores, boxes, classes = None
    
    ### END CODE HERE ###
    
    return scores, boxes, classes

In [ ]:
with tf.Session() as test_b:
    yolo_outputs = (tf.random_normal([19, 19, 5, 1], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 2], mean=1, stddev=4, seed = 1),
                    tf.random_normal([19, 19, 5, 80], mean=1, stddev=4, seed = 1))
    scores, boxes, classes = yolo_eval(yolo_outputs)
    print("scores[2] = " + str(scores[2].eval()))
    print("boxes[2] = " + str(boxes[2].eval()))
    print("classes[2] = " + str(classes[2].eval()))
    print("scores.shape = " + str(scores.eval().shape))
    print("boxes.shape = " + str(boxes.eval().shape))
    print("classes.shape = " + str(classes.eval().shape))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>scores[2]</b>
        </td>
        <td>
           138.791
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes[2]</b>
        </td>
        <td>
           [ 1292.32971191  -278.52166748  3876.98925781  -835.56494141]
        </td>
    </tr>
    <tr>
        <td>
            <b>classes[2]</b>
        </td>
        <td>
           54
        </td>
    </tr>
        <tr>
        <td>
            <b>scores.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
    <tr>
        <td>
            <b>boxes.shape</b>
        </td>
        <td>
           (10, 4)
        </td>
    </tr>
    <tr>
        <td>
            <b>classes.shape</b>
        </td>
        <td>
           (10,)
        </td>
    </tr>
</table>

## Summary for YOLO :

- 입력 이미지는 (608, 608, 3) 의 shape를 가집니다.
- 이 이미지는 CNN 신경망을 통과해 (19, 19, 5, 85) 차원의 출력으로 변환됩니다.
- 해당 출력의 마지막 두 차원을 평면화시키면, 최종 출력층의 shape는 (19, 19, 425)가 됩니다.
  - 19x19 그리드의 각 셀은 425개의 숫자를 가지고 있습니다.
  - 각 셀마다 5개 anchor boxes에 해당하는 경계 상자를 가지고 있습니다.
  - 각 경계 상자는 ($p_c, b_x, b_y, b_h, b_w$)의 총 5개의 데이터에, 탐지해야 하는 클래스의 개수 총 80개를 더한 총 85개의 데이터를 가지고 있습니다.
- 이후 아래 규칙에 맞게 몇 개의 경계 상자를 선택합니다.
  - Score-thresholding : 특정 임계치보다 작은 점수를 가진 클래스를 감지한 경계 상자를 제거합니다.
  - Non-max suppression : IoU를 계산하고, 서로 겹치는 경계 상자를 제거합니다.
- 최종적인 YOLO 알고리즘의 결과를 출력합니다.

## 3 - Test YOLO pre-trained model on images

이 부분에서는 사전 학습 된 모델을 사용하여 자동차 감지 데이터 세트에서 테스트합니다. 계산 그래프를 실행하고 텐서를 평가하려면 세션이 필요합니다.

In [ ]:
sess = K.get_session()

### 3.1 - Defining classes, anchors and image shape


* 80 개의 클래스를 탐지하려고하고 5 개의 anchor box를 사용하고 있음을 기억하세요
* "coco_classes.txt"와 "yolo_anchors.txt" 두 파일에 80 개의 클래스와 5 개의 상자에 대한 정보를 수집했습니다.
* 텍스트 파일에서 클래스 이름과 anchor를 읽습니다.
* 자동차 감지 데이터 세트에는 720x1280 이미지가 있으며,이를 608x608 이미지로 전처리 했습니다.

In [ ]:
class_names = read_classes("model_data/coco_classes.txt")
anchors = read_anchors("model_data/yolo_anchors.txt")
image_shape = (720., 1280.)    

### 3.1 - Loding a pre-trained model

* YOLO 모델을 학습하는 데는 매우 오랜 시간이 걸리며 광범위한 클래스에 대해 라벨이 지정된 경계 상자를 담고 있는 상당히 큰 데이터 세트가 필요합니다.
* "yolo.h5"에 저장된 기존의 사전 훈련 된 Keras YOLO 모델을 로드합니다.
* 이 가중치는 공식 YOLO 웹 사이트에서 가져온 것이며 Allan Zelener가 작성한 함수를 사용하여 변환되었습니다. 해당 참고 문헌은 이 노트의 끝에 있습니다. 기술적으로는 "YOLOv2"모델의 매개 변수이지만 이 노트북에서는 간단히 "YOLO"라고합니다.

이 파일에서 모델을 로드하려면 아래 셀을 실행하세요.

In [ ]:
yolo_model = load_model("model_data/yolo.h5")

위 코드는 훈련 된 YOLO 모델의 가중치를 로드합니다. 다음은 모델에 포함 된 레이어의 요약입니다.

In [ ]:
yolo_model.summary()

**참고** : 일부 컴퓨터에서는 Keras의 경고 메시지가 표시 될 수 있습니다. 걱정하지 마세요. 괜찮습니다.

**알림** : 이 모델은 그림(2) 에 설명 된대로 전처리된 (m, 608, 608, 3) shape의 입력 이미지 배치를 (m, 19, 19, 5, 85) shape의 텐서 로 변환합니다.

### 3.3 - Convert output of the model to usable bounding box tensors

`yolo_model`의 출력은 (m, 19, 19, 5, 85)의 텐서로, non-trival 처리와 conversion 연산이 필요합니다. 아래 코드 블록이 해당 로직을 수행합니다.


`yolo_head`가 어떻게 구현되는지 궁금하다면 ['keras_yolo.py'](https://github.com/allanzelener/YAD2K/blob/master/yad2k/models/keras_yolo) 파일에서 함수 정의를 찾을 수 있습니다. 파일은 `yad2k/models/keras_yolo.py` 경로의 작업 공간에 있습니다.

In [ ]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))

그래프에`yolo_outputs`를 추가 했습니다. 이 4 개의 텐서 세트는 `yolo_eval` 함수의 입력으로 사용할 준비가 되었습니다.

### 3.4 - Filtering boxes

`yolo_outputs`는 `yolo_model`의 모든 경계 상자를 올바른 형식으로 제공했습니다. 이제 필터링을 수행하고 최적의 경계 상자만 선택할 준비가 되었습니다. 이제 이전에 구현한 `yolo_eval`을 호출 해 보겠습니다.

In [ ]:
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

### 3.5 - Run the graph on an image

본격적으로 추론을 시작해봅시다. 여러분은 아래 3단계로 요악할 수 있는 그래프를 만들었습니다.
1. `yolo_model.input`은 주어진 `yolo_model`입니다. 이 모델은 `yolo_model.output`을 계산하기 위해서 사용됩니다.
2. `yolo_model.output`은 `yolo_head`에 의해 변환됩니다. 해당 함수는 `yolo_outputs`을 리턴합니다.
3. `yolo_outputs`은 필터링 함수인 `yolo_eval`를 통과합니다. 이후 `scores`, `boxes`, `classes`를 리턴합니다.

**연습 문제** : YOLO 알고리즘 테스트하기 위해 그래프를 실행하는 `predict()` 를 구현합니다. `scores`, `boxes`, `classes`를 계산하려면 TensorFlow 세션을 실행해야합니다.

아래 코드는 다음 기능도 사용합니다.
```python
image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))
```
이 값은, 아래와 같은 return value를 갖습니다.
- 이미지 : 상자를 그리는 데 사용되는 이미지의 파이썬 (PIL) 표현. 사용할 필요가 없습니다.
- image_data : 이미지를 나타내는 numpy-array. 이것은 CNN에 대한 입력이 될 것입니다.


#### Hint: Using the TensorFlow Session object
* 앞서 우리는 `K.get_Session()` 을 호출하고 `sess` 에 Session 객체를 저장했습니다.
* 텐서 목록을 평가하기 위해 다음과 같이 `sess.run()`을 호출합니다.
```
sess.run(fetches=[tensor1,tensor2,tensor3],
         feed_dict={yolo_model.input: the_input_variable,
                    K.learning_phase():0
         }
```

* `scores`, `boxes`, `classes` 는 `predict` 함수로 전달되지 않지만 `predict` 함수 내에서 사용할 전역 변수입니다.

In [ ]:
def predict(sess, image_file):
    """
    Runs the graph stored in "sess" to predict boxes for "image_file". Prints and plots the predictions.
    
    Arguments:
    sess -- your tensorflow/Keras session containing the YOLO graph
    image_file -- name of an image stored in the "images" folder.
    
    Returns:
    out_scores -- tensor of shape (None, ), scores of the predicted boxes
    out_boxes -- tensor of shape (None, 4), coordinates of the predicted boxes
    out_classes -- tensor of shape (None, ), class index of the predicted boxes
    
    Note: "None" actually represents the number of predicted boxes, it varies between 0 and max_boxes. 
    """

    # Preprocess your image
    image, image_data = preprocess_image("images/" + image_file, model_image_size = (608, 608))

    # Run the session with the correct tensors and choose the correct placeholders in the feed_dict.
    # You'll need to use feed_dict={yolo_model.input: ... , K.learning_phase(): 0})
    ### START CODE HERE ### (≈ 1 line)
    out_scores, out_boxes, out_classes = None
    ### END CODE HERE ###

    # Print predictions info
    print('Found {} boxes for {}'.format(len(out_boxes), image_file))
    # Generate colors for drawing bounding boxes.
    colors = generate_colors(class_names)
    # Draw bounding boxes on the image file
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    # Save the predicted bounding box on the image
    image.save(os.path.join("out", image_file), quality=90)
    # Display the results in the notebook
    output_image = scipy.misc.imread(os.path.join("out", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

**모범 답안**:

<table>
    <tr>
        <td>
            **Found 7 boxes for test.jpg**
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.60 (925, 285) (1045, 374)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.66 (706, 279) (786, 350)
        </td>
    </tr>
    <tr>
        <td>
            **bus**
        </td>
        <td>
           0.67 (5, 266) (220, 407)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.70 (947, 324) (1280, 705)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.74 (159, 303) (346, 440)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.80 (761, 282) (942, 412)
        </td>
    </tr>
    <tr>
        <td>
            **car**
        </td>
        <td>
           0.89 (367, 300) (745, 648)
        </td>
    </tr>
</table>

방금 실행 한 모델은 실제로 "coco_classes.txt"에 나열된 80 개의 서로 다른 클래스를 감지 할 수 있습니다. 자신의 이미지에서 모델을 테스트하려면 :
    
  1. 이 노트북의 상단 표시 줄에서 "파일"을 클릭 한 다음 "열기"를 클릭하여 Coursera Hub로 이동합니다.
  2. "images"폴더에있는 Jupyter Notebook의 디렉토리에 이미지를 추가합니다.
  3. 코드 위의 셀에 이미지 이름을 씁니다.
  4. 코드를 실행하고 알고리즘의 출력을 확인하십시오!

모든 이미지에 대해 for 루프에서 세션을 실행하는 경우. 다음과 같은 결과를 얻을 수 있습니다.

<video width="400" height="200" src="arts/pred_video_compressed2.mp4" type="video/mp4" controls>
</video>

## What you should remember:

- YOLO는 빠르고 정확한 최첨단 물체 감지 모델입니다.
- 19x19x5x85 차원의 볼륨을 출력하는 CNN을 통해 입력 이미지를 실행합니다.
- 인코딩은 각각의 19x19 셀에 5 개의 상자에 대한 정보가 포함 된 그리드로 볼 수 있습니다.
- Non-max suppression를 사용하여 모든 상자를 필터링합니다. 구체적으로 특별히:
  - 높은 확률의 상자 만 유지하기 위한 class score 임계 값
  - 겹치는 상자를 제거하기위한 IoU (Intersection over Union) 임계 값
- 무작위로 초기화 된 가중치로 YOLO 모델을 훈련하는 것은 많은 계산과 함께 대규모 데이터 세트가 필요하기 때문에 이 연습에서는 이전에 훈련 된 모델 파라미터를 사용했습니다. 원하는 경우 자신의 데이터 세트로 YOLO 모델을 미세 조정 해 볼 수도 있지만, 별로 추천하지 않습니다.

**References**: The ideas presented in this notebook came primarily from the two YOLO papers. The implementation here also took significant inspiration and used many components from Allan Zelener's GitHub repository. The pre-trained weights used in this exercise came from the official YOLO website. 
- Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi - [You Only Look Once: Unified, Real-Time Object Detection](https://arxiv.org/abs/1506.02640) (2015)
- Joseph Redmon, Ali Farhadi - [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) (2016)
- Allan Zelener - [YAD2K: Yet Another Darknet 2 Keras](https://github.com/allanzelener/YAD2K)
- The official YOLO website (https://pjreddie.com/darknet/yolo/) 

**Car detection dataset**:

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/">

<img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>
<br>
<span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">The Drive.ai Sample Dataset</span> (provided by drive.ai) is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. We are grateful to Brody Huval, Chih Hu and Rahul Patel for  providing this data. 